In [1]:
import cifar10
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [2]:
cifar10.data_path = "data/CIFAR-10/"

In [3]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [4]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [5]:
x_images_training=images_train[:, :, :, 0]
# x_train=x_images_training.reshape((50000, 32*32))
x_train=images_train.reshape(50000,3072)
x_test=images_test.reshape(10000,3072)
y_train=cls_train
x_images_test=images_test[:, :, :, 0]
# x_test=x_images_test.reshape((10000, 32*32))
y_test=cls_test
x_train.shape,y_train.shape

((50000, 3072), (50000,))

**Before Applying SVC we need to apply PCA to get useful features only**

In [6]:
pc=PCA()
pc.fit(x_train)

PCA()

In [7]:
total=sum(pc.explained_variance_)
cSum=0
k=0
while(cSum/total<0.95):
    cSum+=pc.explained_variance_[k]
    k=k+1
k    

217

In [8]:
pca=PCA(n_components=k,whiten=True)
x_train_pca=pca.fit_transform(x_train)
x_test_pca=pca.transform(x_test)
x_train_pca.shape,y_train.shape

((50000, 217), (50000,))

In [9]:
svc=SVC()
grid1={"C":[1e2,1e4,1e1,5e3,1e3,5e2],"gamma":[1e-3,1e-2,5e-3,5e-2,1e-4,1e-5]}
abc=GridSearchCV(svc,grid1,n_jobs=-1)

In [14]:
abc.fit(x_train_pca,y_train)

GridSearchCV(estimator=SVC(), n_jobs=2,
             param_grid={'C': [100.0, 10000.0, 10.0, 5000.0, 1000.0, 500.0],
                         'gamma': [0.001, 0.01, 0.005, 0.05, 0.0001, 1e-05]})

In [16]:
abc.best_estimator_

SVC(C=100.0, gamma=0.005)

In [52]:
clf=SVC(kernel='rbf',C=10000,gamma=0.0005)
clf.fit(x_train_pca[0:20000,:],y_train[0:20000])
y_pred=clf.predict(x_test_pca)
accuracy_score(y_test,y_pred)

0.4935

In [47]:
# abc.score(x_test_pca,y_test)
accuracy_score(y_test,y_pred)

0.5088

In [48]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.56      0.56      0.56      1000
           1       0.54      0.66      0.59      1000
           2       0.40      0.39      0.39      1000
           3       0.36      0.38      0.37      1000
           4       0.48      0.41      0.44      1000
           5       0.43      0.40      0.42      1000
           6       0.58      0.53      0.56      1000
           7       0.58      0.56      0.57      1000
           8       0.66      0.60      0.63      1000
           9       0.50      0.60      0.54      1000

    accuracy                           0.51     10000
   macro avg       0.51      0.51      0.51     10000
weighted avg       0.51      0.51      0.51     10000



In [49]:
dt=np.dtype('U25')
output=np.empty((10000),dtype=dt) 
for i in range(len(y_pred)):
    output[i]=class_names[y_pred[i]]

In [50]:
np.savetxt("answers.csv", output,fmt='%s')